In [1]:
import keras

import os
import re
import json
import subprocess
import shlex
import string
import csv
import pandas as pd
import numpy

from keras.models import model_from_json

import txtfileprocessing as txtp

Using TensorFlow backend.


In [6]:
def symbolizeAndVectorizeToFile(filevectors):
    randomfile = "/home/(username)/Workfiles/goodvectors2.txt"
    rf = open(randomfile,"a+")
    #rf.write(json.dumps(numdict))
    for i in range(0,len(filevectors)-1):
        rf.write("%s," % str(filevectors[i]))
    rf.write(str(filevectors[len(filevectors)-1]))
    rf.write("\n")
    #data = json.load(rf)
    rf.close()

def symbolizeAndVectorize(filetoopen):
    keywordsfile = "keywordsdict.txt"
    kfdread = open(keywordsfile,"r+")
    keywords = json.load(kfdread)
    
    content = []
    opcodes = []
    filevectors = []
    with open(filetoopen) as f:
        content = f.readlines()
        content = [x.strip() for x in content]
        #content - an array of lines from the file that was opened
    for line in content:
        if not ("\t" in line):
            continue
        templist = line.split("\t")
        tempstr = templist[2]
        tempstr = " ".join(tempstr.split())
        templist = re.split(',',tempstr)
        for i in range(0,len(templist)):
            try:
                hexnum = int(templist[i],16)
                templist[i] = "num"
            except ValueError:
                continue
        templist2 = templist[0].split()
        opcodestr = templist2.pop(0)
        opcodes.append(opcodestr)
        
        tempstr2 = " ".join(templist2)
        templist[0] = tempstr2
        templist.insert(0,opcodestr)
        operand1 = templist[1].split()
        
        operand2 = []
        try:
            operand2 = templist[2].split()
        except IndexError:
            operand2.append('')
        lines = opcodes+operand1+operand2
        opcodes = []
        
        for i in range(0,len(lines)):
            if str(lines[i]).startswith("<"):
                lines[i] = "num"
            elif str(lines[i]).startswith("["):
                lines[i] = "num"
            try:
                hexnum = int(lines[i],16)
                lines[i] = "num"
            except ValueError:
                continue
        
        for j in range(len(lines)):
            if lines[j] in keywords:
                lines[j] = keywords[lines[j]]
            if lines[j] != '':
                filevectors.append(lines[j])
        
        
    kfdread.close()
    symbolizeAndVectorizeToFile(filevectors)

In [7]:
def vectorizeFolder():
    functype = "good"
    #for path, dirs, files in os.walk(folderpath):
    for path, dirs, files in os.walk("/home/(username)/Workfiles/TrainingFiles"):
        for file in files:
            if(functype in file):
                fullfilepath = path+"/"+file
                symbolizeAndVectorize(filetovectorize)
                
vectorizeFolder()

In [17]:
def LoadModel():
    # load json and create model
    json_file = open('model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("model.h5")
    print("Loaded model from disk")
    return loaded_model

def GetFunctions(file,path):
    functions = []
    processArg = "readelf -SW "+file
    args = shlex.split(processArg)
    output = subprocess.Popen(args,stdout=subprocess.PIPE,cwd=path ).communicate()[0]
    out = output.decode('utf-8').split('\n')
    for item in out:
        if(item.startswith(".text.",7)):
            item = item[7:]
            newitem = item[6:item.find(' ')]
            functions.append(newitem)
    return functions

def GenerateAssemCodeFiles(functions,file,path):
    localpath = "/home/(username)/Workfiles/TrainingFiles/"+file[:-2]
    if not(os.path.exists(localpath)):
        folderprocessArg = "mkdir "+file[:-2]
        folderargs = shlex.split(folderprocessArg)
        try:
            folderout = subprocess.check_output(folderargs,cwd="/home/(username)/Workfiles/TrainingFiles")
        except subprocess.CalledProcessError:
            print("error while creating folder")
    filenames = []
    for function in functions:
        processArg = "objdump -M intel -w -d --section=.text."+function+" "+file
        args = shlex.split(processArg)
        try:
            out = subprocess.check_output(args,cwd=path)
            filename = function+".txt"
            filenames.append(filename)
            filename = localpath+"/"+filename
            #original path+ new folder+ new file's name
            outfile = open(filename,'w')
            outfile.write(out.decode("utf-8"))
            outfile.close()
        except subprocess.CalledProcessError:
            print("error while creating .txt file")
    return filenames,localpath

def CreateTxtFile(newpath):
    for path, dirs, files in os.walk(newpath):
        for file in files:
            symbolizeAndVectorize(file,path)

#Function to pad zeroes
def trp(l, n):
    return l[:n] + [0]*(n-len(l))

def CreateCsvFile(newpath):
    total = 0
    count = 0
    fixedsize = 200
    with open(newpath+"/"+"vector.txt","r") as filletbread:
        lines = [line.strip() for line in filletbread]
        with open(newpath+"/"+"vector.csv","w") as out_file:
            codewriter = csv.writer(out_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            for line in lines:
                templist = [int(x.strip()) for x in line.split(',')]
                
                #Make the size of templist to be fixedsize
                if (len(templist)<fixedsize):
                    templist = trp(templist,fixedsize)
                elif (len(templist)>fixedsize):
                    templist = templist[:fixedsize]
                
                codewriter.writerow(templist)

def EvalCsv(newpath):
    keywordsfile = "keywordsdict.txt"
    kfdread = open(keywordsfile,"r")
    keywords = json.load(kfdread)
    kfdread.close()
    n_labels = len(keywords)+2
    model = LoadModel()
    testset = pd.read_csv(newpath+"/"+"vector.csv",header=None)
    testx = testset.iloc[:, 0:].values

    testx1 = numpy.eye(n_labels)[testx]

    testy = model.predict_classes(testx1)
    probability = model.predict_proba(testx1)

    ysum = 0
    probsum = 0
    count=0
    for i in range(0,len(testy)):
        count+=1
        ysum += testy[i]
        probsum += probability[i]
        print("X=%s, Predicted=%s" % (testx1[i], testy[i]))
        print("Probability=%s"%(probability[i]))

def ProcessSelectedFile(filepath):
    pathlist = filepath.split("/")
    file = pathlist[-1]
    pathlist2 = pathlist[1:-1]
    path = '/'.join(pathlist2)
    path = '/'+path
    #print(file)
    #print(path)
    functions = []
    if(file.endswith(".o")):
        #functype = file[:-2]+"_bad"
        #functions.append("main") #To retrieve functype dynamically
        functions = GetFunctions(file,path)
    else:
        print("Please select an object file")
    """with open('CodeProcessor.ipynb','r') as codeprocessorfile:
        codeprocessorread = codeprocessorfile.read()
    codeprocessor = json.loads(codeprocessorread)
    #Prepare a cell to process selected file
    code_to_execute = compile("\n".join(codeprocessor["cells"][1]["source"]),"<string>","exec")
    exec(code_to_execute)"""
    return functions,file,path

In [18]:
for path, dirs, files in os.walk("/home/(username)/Desktop/cexamples/testcases/000"):
    for file in files:
        #functype = file[:-2]+"_bad"
        functype = "main"
        if(file.endswith(".o")):
            filepath = os.path.join(path, file)
            functions,file1,path1 = ProcessSelectedFile(filepath)
            filenames,newpath = GenerateAssemCodeFiles(functions,file1,path1)
            #CreateTxtFile(newpath)
            #CreateCsvFile(newpath)

In [ ]:
file = "newfolder.o"

localpath = "/home/(username)/Workfiles/TrainingFiles/"+file[:-2]
print(localpath)
if not(os.path.exists(localpath)):
    print("In here!")
    folderprocessArg = "mkdir "+file[:-2]
    folderargs = shlex.split(folderprocessArg)
    try:
        folderout = subprocess.check_output(folderargs,cwd="/home/(username)/Workfiles/TrainingFiles")
    except subprocess.CalledProcessError:
        print("error while creating folder")

In [21]:
def printfile(filetoprint):
    keywordsfile = "keywords.txt"
    kfread = open(keywordsfile,"r+")
    keywords = kfread.read().splitlines()
    keywords = list(filter(None, keywords))
    kwlistsize = len(keywords)
    #Reads the contents of the filetoprint, and store as an array
    content = []
    with open(filetoprint) as f:
        content = f.readlines()
        content = [x.strip() for x in content]
    #Check that for each line in content, opcodes that are not already in the opcodes array will be appended,
    #and other words that are not registers or hexadecimal numbers will be added to instructions array.
    for line in content:
        if not ("\t" in line):
            continue
        templist = line.split("\t")
        tempstr = templist[2]
        tempstr = " ".join(tempstr.split())
        templist = re.split(' |,',tempstr)
        tempstr = templist[0]
        if not(tempstr in keywords):
            keywords.append(tempstr)
        for i in range(1,len(templist)):
            if not (templist[i] in keywords):
                if not ((all(c in string.hexdigits for c in templist[i]))or(templist[i].startswith("<"))or(templist[i].startswith("0x"))):
                    for j in keywords:
                        if (j in templist[i]):
                            break
                    else:
                        keywords.append(templist[i])
    #Write opcodes array to opcodes file, write instructions array to instructions file
    #Closing 2 files after writing
    inputstring = ""
    for i in range(kwlistsize,len(keywords)):
        inputstring = inputstring + keywords[i] + "\n"
    kfread.write(inputstring)
    kfread.close()
    
def printFolder():      
    for path, dirs, files in os.walk("/home/(username)/Workfiles/TrainingFiles"):
        for file in files:
            if(file.endswith(".txt")):
                filetoprint = path+"/"+file
                printfile(filetoprint)
                
#printFolder()

def filetodictionary():
    kwdict = {}
    keywordsfile = "keywords.txt"
    kfread = open(keywordsfile,"r")
    keywords = kfread.read().splitlines()
    keywords = list(filter(None,keywords))
    kwlistsize = len(keywords)
    kfread.close()
    for i in range(0,kwlistsize):
        kwdict[keywords[i]] = i+1
    keywordsfile = "keywordsdict.txt"
    kfdread = open(keywordsfile,"w")
    kfdread.write(json.dumps(kwdict))
    kfdread.close()
    
filetodictionary()    

In [ ]:
keywordsfile = "keywordsdict.txt"
kfdread = open(keywordsfile,"r")
keywords = json.load(kfdread)
kfdread.close()

